# Chương I

# Natural Language Processing LÀ GÌ?

NLP - hay Xử lý ngôn ngữ tự nhiên - là viết tắt của một loạt các kỹ thuật được thiết kế để giúp máy học từ văn bản. Xử lý ngôn ngữ tự nhiên hỗ trợ mọi thứ từ chatbot đến công cụ tìm kiếm và được sử dụng trong nhiều tác vụ khác nhau như phân tích tình cảm và dịch máy.

## 1/ Thư viện cơ bản cần

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

## 2/ Truy suất xem file data(csv)

In [2]:
import os
for dirname, _, filenames in os.walk('D:/Nhom15DHIOT17A/Codes/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:/Nhom15DHIOT17A/Codes/data\sample_submission.csv
D:/Nhom15DHIOT17A/Codes/data\test.csv
D:/Nhom15DHIOT17A/Codes/data\test_predictions.csv
D:/Nhom15DHIOT17A/Codes/data\train.csv


## 3/ Đọc data

In [3]:
train_df = pd.read_csv("D:/Nhom15DHIOT17A/Codes/data/train.csv")
test_df = pd.read_csv("D:/Nhom15DHIOT17A/Codes/data/test.csv")


## 4/ Phân loại cơ bản đâu là thảm họa và đâu không phải là thảm họa

### Không phải thảm họa

In [4]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

### Thảm họa

In [5]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
train_df[train_df["target"]==1]["text"].values

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

### Trong bộ dữ liệu này , cột target dùng để phân loại tweet
   #### 1.target = 0: Tweet không liên quan đến thảm họa, thường chỉ đề cập đến các hoạt động hoặc cảm xúc bình thường, không mang tính chất nguy hiểm. Ví dụ: "What's up man?", "I love fruits".

   #### 2.target = 1: Tweet liên quan đến thảm họa, thường mô tả sự cố hoặc thiên tai thực sự có thể gây nguy hiểm. Ví dụ: "Forest fire near La Ronge Sask. Canada".

## 5/ Xây dựng vecto

### Sau khi phân loại được đâu là tweet thảm họa đâu là tweet không phải thảm họa.Nhóm sử dụng scikit-learn CountVectorizer để đếm xem số lượng từ trong mỗi tweet và chuyển thành dữ liệu máy học.

### Với 5 tweet đầu tiên

In [7]:
# Khởi tạo CountVectorizer
count_vectorizer = feature_extraction.text.CountVectorizer()

# Đếm số từ trong 5 tweet đầu tiên trong dữ liệu
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

# In nội dung của 5 tweet đầu tiên
print("Nội dung của 5 tweet đầu tiên:")
print(train_df["text"][0:5].values)

# In danh sách tất cả các từ đã được trích xuất từ các tweet đầu tiên (từ vựng)
print("\nDanh sách tất cả các từ đã được trích xuất:")
print(count_vectorizer.get_feature_names_out())

# In từ vựng và chỉ số tương ứng
print("\nTừ vựng và chỉ số tương ứng:")
print(count_vectorizer.vocabulary_)

# In ma trận vector chi tiết của 5 tweet đầu tiên
print("\nMa trận vector chi tiết của 5 tweet đầu tiên:")
rows, cols = example_train_vectors.nonzero()  # Lấy chỉ số hàng và cột của ma trận không rỗng
for r, c in zip(rows, cols):
    print(f"({r}, {c})\t{example_train_vectors[r, c]}")



Nội dung của 5 tweet đầu tiên:
['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
 'Forest fire near La Ronge Sask. Canada'
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"
 '13,000 people receive #wildfires evacuation orders in California '
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ']

Danh sách tất cả các từ đã được trích xuất:
['000' '13' 'alaska' 'all' 'allah' 'are' 'as' 'asked' 'being' 'by'
 'california' 'canada' 'deeds' 'earthquake' 'evacuation' 'expected' 'fire'
 'forest' 'forgive' 'from' 'got' 'in' 'into' 'just' 'la' 'may' 'near' 'no'
 'notified' 'of' 'officers' 'or' 'orders' 'other' 'our' 'people' 'photo'
 'place' 'pours' 'reason' 'receive' 'residents' 'ronge' 'ruby' 'sask'
 'school' 'sent' 'shelter' 'smoke' 'the' 'this' 'to' 'us' 'wildfires']

Từ vựng và chỉ số tương ứng:
{'our': 34, 'deeds': 12, 'are': 5, 'the': 49, 're

In [8]:

# Dòng sau đây sẽ in ra kích thước của vector đầu tiên
print(example_train_vectors[0].todense().shape)
# Dòng sau đây sẽ in ra toàn bộ giá trị của vector đầu tiên dưới dạng ma trận dày đặc
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


### Giải thích ý nghĩa về 5 tweet đầu tiên
##### 1.Có 54 từ trong 5 tweet đầu tiên
##### 2.Tweet đầu tiên chỉ chứa một số mã thông báo duy nhất đó - tất cả các số khác không ở trên là các mã thông báo TỒN TẠI trong tweet đầu tiên.

## 6/Thực hiện kiểm tra chéo

### Mục tiêu của việc kiểm tra chéo
##### 1.kiểm tra mô hình và đánh giá hiệu suất trên dữ liệu huấn luyện
##### 2.Phương pháp sử dụng cross-validation chia dữ liệu thành nhiều phần huấn luyện mô hình trên một phần và kiểm tra các phần còn lại và lặp lại quá trình nhiều lần
##### 3.Sử dụng F1 score làm chỉ số đánh giá

In [9]:
from sklearn import model_selection, linear_model
from sklearn.feature_extraction.text import CountVectorizer

# Khởi tạo CountVectorizer
count_vectorizer = CountVectorizer()

# Chia dữ liệu thành các vector đặc trưng
train_vectors = count_vectorizer.fit_transform(train_df["text"])

# Chuyển đổi dữ liệu kiểm tra mà không sử dụng .fit_transform()
# Đảm bảo rằng các token trong train_vectors là duy nhất
test_vectors = count_vectorizer.transform(test_df["text"])

# Khởi tạo mô hình phân loại Ridge
clf = linear_model.RidgeClassifier()

# Thực hiện kiểm tra chéo để đánh giá mô hình
scores = model_selection.cross_val_score(
    clf,  # Mô hình phân loại đã được định nghĩa
    train_vectors,  # Ma trận vector đặc trưng của dữ liệu huấn luyện
    train_df["target"],  # Nhãn mục tiêu cho dữ liệu huấn luyện
    cv=3,  # Số lượng phần (folds) để chia dữ liệu (3 phần ở đây)
    scoring="f1"  # Sử dụng F1 score làm chỉ số đánh giá
)

# In ra các điểm F1 cho từng lần kiểm tra chéo
print("Điểm F1 cho từng lần kiểm tra chéo:", scores)

# Tính và in ra điểm F1 trung bình
print("Điểm F1 trung bình:", scores.mean())

Điểm F1 cho từng lần kiểm tra chéo: [0.59453669 0.5642787  0.64082434]
Điểm F1 trung bình: 0.5998799106916805


In [10]:
clf.fit(train_vectors, train_df["target"])
sample_submission = pd.read_csv("D:/Nhom15DHIOT17A/Codes/data/sample_submission.csv")
sample_submission["target"] = clf.predict(test_vectors)
sample_submission.head()


,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
